In [1]:
import pandas as pd
import sqlite3
pd.set_option('display.max_columns', None)


### yeni verinin eskisiyle aynı olup olmadığını kontrol edildi

In [3]:
PPA_new = pd.read_excel("USA_12.6.22.xlsx", dtype=str)
PPA_old = pd.read_excel("USA_version2.xlsx", dtype=str)

In [4]:
# check if two columns of a dataframe are equal
(PPA_new['BIOID'] == PPA_old['BIOID']).all()

True

### iki veri birleştirildi

burada dsp ve github verisi birleştirildi. code sütunu elle oluşturuldu. 

daha sonra sqlite'a aktarıldı. orada buna göre twitlerin filtrelenmesi yapıldı.

filtrelenen twitlerden hashtagler çekildi ve chatGPT ile kategorileri oluşturuldu

In [20]:
import pandas as pd

# Load and preprocess the PPA dataset
PPA = pd.read_excel('dsp.xlsx', dtype=str)
PPA = PPA.applymap(lambda x: x.strip() if isinstance(x, str) else x)
state_abbreviations = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maryland': 'MD',
    'Missouri': 'MO',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Washington': 'WA',
    'Wisconsin': 'WI'
}
PPA['state_abb'] = PPA['OFFICE1_CONSTITUENCYLEVEL2'].map(state_abbreviations)
PPA = PPA[
    (PPA['OFFICE1_LEVEL'] == "1") & 
    (PPA['PARTY NAME, ENGLISH'].isin(['Democrat', 'Democrat/NPL', 'Democrat/Working Families', 'Republican', 'Republican/Conservative'])) &
    (PPA['OFFICE1'].isin(['U.S. Senate', 'US Senate', 'U.S. Senator'])) &
    (PPA['OFFICE1_ELECTION_RESULT_GENERAL'].isin(['0', '1']))
]
print(len(PPA))

# Filter the PPA dataset to include only winners
PPA_winners = PPA[PPA['OFFICE1_ELECTION_RESULT_GENERAL'] == "1"]
print(len(PPA_winners))

PPA_losers = PPA[PPA['OFFICE1_ELECTION_RESULT_GENERAL'] == "0"]
print(len(PPA_losers))

# Load and preprocess the github dataset
github = pd.read_excel('githubFixed.xlsx', dtype=str)
github = github[github["type"] == "sen"]
github = github.applymap(lambda x: x.strip() if isinstance(x, str) else x)
print(len(github))


72
35
37
100


In [21]:
# Merge the filtered PPA dataset with the github dataset using full names
merged_data_fullname_winners = pd.merge(PPA_winners, github, 
                                        left_on='code', 
                                        right_on='code', 
                                        how='outer')

# Check the length of the resulting dataset after merging with full names
len(merged_data_fullname_winners)

100

In [22]:
result = pd.concat([merged_data_fullname_winners, PPA_losers], axis=0, ignore_index=True)
len(result)

137

In [23]:
result = result.rename(columns={'gender': 'gender2'})

In [14]:
# connect to sqlite database
conn = sqlite3.connect("D:/Dropbox/projeler/ndsu_polarization/twitterVeriToplama/twitter004.db")

# Convert all columns in the result DataFrame to string type
result = result.astype(str)

# send results to sqlite database
result.to_sql('senatorlist', conn, if_exists='replace', index=False)

# close connection
conn.close()


### hastaglerden matrix

In [6]:
import pandas as pd
import requests
import io

SHEET_ID = '1mIWOqvpUA2JvRGeTuXO017zG4lfGPa4JcA1AEAOK9S8'

SHEET_NAME = 'unsup_01'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
hashtags = pd.read_csv(url)
hashtags = hashtags[['Hashtag', 'Category']]
hashtags

,Hashtag,Category
0,AmericanRescuePlan,Economic & Infrastructure Issues
1,Bidenflation,Economic & Infrastructure Issues
2,Bidenomics,Economic & Infrastructure Issues
3,BipartisanInfrastructureLaw,Economic & Infrastructure Issues
4,CancelStudentDebt,Economic & Infrastructure Issues
...,...,...
202,TX38,States and Local Politics
203,TX15,States and Local Politics
204,WASen,States and Local Politics
205,wisen,States and Local Politics


### upload birleştirilmiş hastag table to sqlite

In [3]:
#read excel file
df = pd.read_excel('all_hashtags.xlsx', dtype=str)

c:\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
df

,#,hashtag,cluster,frequency,unique_users
0,1,IASen,IASEN,186,3
1,2,NHSEN,NHSEN,127,1
2,3,HurricaneIan,HuracanIan,126,7
3,4,ElectionDay,ElectionDay,126,30
4,5,RonsRoadTour,RonsRoadTour,118,1
...,...,...,...,...,...
1959,1960,20Counties,20Counties,1,1
1960,1961,2023FarmBill,FarmBill,1,1
1961,1962,2022GWSummit,2022GWSummit,1,1
1962,1963,2022CAGeneralElection,2022CAGeneralElection,1,1


In [5]:
# connect to sqlite database
conn = sqlite3.connect("D:/Dropbox/projeler/ndsu_polarization/twitterVeriToplama/twitter004.db")

# send results to sqlite database
df.to_sql('all_hashtags', conn, if_exists='replace', index=False)

# close connection
conn.close()

In [1]:
import sqlite3
import json

# Connect to your SQLite database
conn = sqlite3.connect("D:/Dropbox/projeler/ndsu_polarization/twitterVeriToplama/twitter004.db")
cursor = conn.cursor()


In [ ]:
# Fetch entities from the tweets table
cursor.execute("SELECT OFFICE2, entities FROM tumTweets2")
rows = cursor.fetchall()

for row in rows:
    OFFICE2 = row[0]
    entities_data = row[1]

    # Check if entities_data is not None before parsing
    if entities_data:
        try:
            entities_json = json.loads(entities_data)
            
            hashtags = entities_json.get('hashtags', [])
            
            for hashtag in hashtags:
                # Extract the actual hashtag text
                hashtag_text = hashtag['tag']
                
                # Insert into the new table
                cursor.execute("INSERT INTO tweet_hashtags2 (OFFICE2, hashtag) VALUES (?, ?)", (OFFICE2, hashtag_text))
        except json.JSONDecodeError:
            print(f"Failed to parse JSON for author_id: {OFFICE2}")

# Commit changes and close connection
conn.commit()
conn.close()


### expert sup için adjacency

In [3]:
SHEET_ID = '1QjDreFrElDv-Edh235ImxsNv-KVArlWItx19tHMKOYI'

SHEET_NAME = 'all_cats_expert_sup'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
hashtags = pd.read_csv(url)
hashtags = hashtags[['cluster', 'sup_category']]
hashtags

,cluster,sup_category
0,AmericanRescuePlan,Economy
1,BidenFlation,Economy
2,Bidenomics,Economy
3,BidenRecession,Economy
4,CancelStudentDebt,Economy
...,...,...
102,DefendWomensRights,SJ Women
103,Roe,SJ Women
104,RoeRoeRoeYourVote,SJ Women
105,RoevemberIsComing,SJ Women


In [4]:
conn = sqlite3.connect("D:/Dropbox/projeler/ndsu_polarization/twitterVeriToplama/twitter004.db")
cursor = conn.cursor()
tweet_hashtags = pd.read_sql_query("SELECT * FROM tweet_hashtags2", conn)
conn.close()

tweet_hashtags


,OFFICE2,hashtag,cluster
0,1,GoDogs,GoDogs
1,1,VeteranServiceRecognitionAct,VeteranServiceRecognitionAct
2,1,ShopSmall,ShopSmall
3,1,TransDayOfRemembrance,TransDayOfRemembrance
4,1,AxeFiles,AxeFiles
...,...,...,...
8026,42,Diwali,Diwali
8027,42,BidenBorderCrisis,BidenBorderCrisis
8028,42,GBO,GBO
8029,42,NationalFarmersDay,NationalFarmersDay


In [9]:
#left join hashtags and tweet_hashtags on cluster
merged_data = pd.merge(tweet_hashtags, hashtags,
                                        left_on='cluster',
                                        right_on='cluster',
                                        how='left')
merged_data

,OFFICE2,hashtag,cluster,sup_category
0,1,GoDogs,GoDogs,NaN
1,1,VeteranServiceRecognitionAct,VeteranServiceRecognitionAct,NaN
2,1,ShopSmall,ShopSmall,Economy
3,1,TransDayOfRemembrance,TransDayOfRemembrance,SJ Identity
4,1,AxeFiles,AxeFiles,NaN
...,...,...,...,...
8026,42,Diwali,Diwali,NaN
8027,42,BidenBorderCrisis,BidenBorderCrisis,Immigration
8028,42,GBO,GBO,NaN
8029,42,NationalFarmersDay,NationalFarmersDay,NaN


In [6]:
# filter where sub_category is not null
merged_data = merged_data[merged_data['sup_category'].notnull()]

merged_data

,OFFICE2,hashtag,cluster,sup_category
2,1,ShopSmall,ShopSmall,Economy
3,1,TransDayOfRemembrance,TransDayOfRemembrance,SJ Identity
5,1,DACA,DACA,Immigration
7,1,RespectForMarriageAct,RespectForMarriage,SJ Identity
8,1,RespectForMarriageAct,RespectForMarriage,SJ Identity
...,...,...,...,...
8017,42,BidenBorderCrisis,BidenBorderCrisis,Immigration
8018,42,BidenInflation,BidenFlation,Economy
8019,42,BidenBorderCrisis,BidenBorderCrisis,Immigration
8022,42,BidenBorderCrisis,BidenBorderCrisis,Immigration


In [12]:
# pivot table cluster and office2
merged_data.pivot_table(index=['sup_category', 'cluster'], aggfunc='size').reset_index()

,sup_category,cluster,0
0,Economy,AmericanRescuePlan,10
1,Economy,BidenFlation,104
2,Economy,BidenRecession,3
3,Economy,Bidenomics,4
4,Economy,CancelStudentDebt,7
...,...,...,...
102,SJ Women,Roevember,9
103,SJ Women,RoevemberIsComing,3
104,SJ Women,SpeakOutAct,4
105,SJ Women,Women,2


In [13]:
# pivot table cluster and office2
merged_data.pivot_table(index=['sup_category'], aggfunc='size').reset_index()

,sup_category,0
0,Economy,311
1,Environment,45
2,Health,157
3,Immigration,118
4,SJ Identity,184
5,SJ Labor,64
6,SJ Women,74


In [22]:
#filter merged_data where sup_category is Economy, Environment, Health, Immigration, SJ Identity, SJ Labor, SJ Women
econ = merged_data[merged_data['sup_category'] == 'Economy']
env = merged_data[merged_data['sup_category'] == 'Environment']
health = merged_data[merged_data['sup_category'] == 'Health']
imm = merged_data[merged_data['sup_category'] == 'Immigration']
sjid = merged_data[merged_data['sup_category'] == 'SJ Identity']
sjlab = merged_data[merged_data['sup_category'] == 'SJ Labor']
sjwom = merged_data[merged_data['sup_category']== 'SJ Women']

In [15]:
econ

,OFFICE2,hashtag,cluster,sup_category
2,1,ShopSmall,ShopSmall,Economy
14,1,DemsCreateJobs,DemsCreateJobs,Economy
53,2,gasprices,gasprices,Economy
54,2,gasprices,gasprices,Economy
57,2,gasprices,gasprices,Economy
...,...,...,...,...
8005,42,Bidenflation,BidenFlation,Economy
8009,42,Bidenflation,BidenFlation,Economy
8011,42,Bidenflation,BidenFlation,Economy
8012,42,Bidenflation,BidenFlation,Economy


In [17]:
import pandas as pd
import numpy as np

df = econ.copy()

# List of unique users
users = df['OFFICE2'].unique()

# Number of unique users
n = len(users)

# Initialize an NxN matrix with zeros
matrix = np.zeros((n, n))

# For each unique hashtag
for cluster in df['cluster'].unique():
    # Get list of users who used this hashtag
    users_with_hashtag = df[df['cluster'] == cluster]['OFFICE2'].tolist()
    
    # Update the matrix for every pair of these users
    for i in range(len(users_with_hashtag)):
        for j in range(i+1, len(users_with_hashtag)):
            # Get matrix indices for the users
            user_i_idx = np.where(users == users_with_hashtag[i])[0][0]
            user_j_idx = np.where(users == users_with_hashtag[j])[0][0]
            
            # Set the matrix entries for this pair of users to 1
            matrix[user_i_idx][user_j_idx] = 1
            matrix[user_j_idx][user_i_idx] = 1  # Symmetrical matrix

matrix


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 1., 1., 1.],
       ...,
       [0., 0., 1., ..., 1., 1., 1.],
       [0., 0., 1., ..., 1., 0., 1.],
       [0., 0., 1., ..., 1., 1., 1.]])

In [ ]:
import pandas as pd
import numpy as np

def generate_adjacency(df):
    # Given OFFICE2 is always between 1-128
    users = np.arange(1, 129)
    
    # Initialize a 128x128 matrix with zeros
    matrix = np.zeros((128, 128))
    
    # For each unique hashtag
    for cluster in df['cluster'].unique():
        # Get list of users who used this hashtag
        users_with_hashtag = df[df['cluster'] == cluster]['OFFICE2'].tolist()
        
        # Update the matrix for every pair of these users
        for i in range(len(users_with_hashtag)):
            for j in range(i+1, len(users_with_hashtag)):
                # Get matrix indices for the users
                user_i_idx = users_with_hashtag[i] - 1  # subtract 1 since array indices start from 0
                user_j_idx = users_with_hashtag[j] - 1
                
                # Set the matrix entries for this pair of users to 1
                matrix[user_i_idx][user_j_idx] = 1
                matrix[user_j_idx][user_i_idx] = 1  # Symmetrical matrix
                
    # Convert the matrix to a DataFrame
    return pd.DataFrame(matrix, index=users, columns=users)

def save_to_sqlite(df, db_name, table_name):
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='replace')
    conn.close()

dfs = [econ, env, health, imm, sjid, sjlab, sjwom]

for i, df in enumerate(dfs, 1):
    adjacency_df = generate_adjacency(df)
    save_to_sqlite(adjacency_df, "adjacency_dbs.db", f"adjacency_df_{i}")


In [28]:
import pandas as pd
import numpy as np
import sqlite3  # Ensure you import sqlite3

def generate_adjacency(df):
    df = df.copy()
    # Convert OFFICE2 column to integers
    df['OFFICE2'] = df['OFFICE2'].astype(int)
    
    # Given OFFICE2 is always between 1-137
    users = np.arange(1, 138)
    
    # Initialize a 128x128 matrix with zeros
    matrix = np.zeros((137, 137))
    
    # For each unique hashtag
    for cluster in df['cluster'].unique():
        print(f"Processing cluster: {cluster}")  # Print current cluster
        
        # Get list of users who used this hashtag
        users_with_hashtag = df[df['cluster'] == cluster]['OFFICE2'].tolist()
        
        # Update the matrix for every pair of these users
        for i in range(len(users_with_hashtag)):
            for j in range(i+1, len(users_with_hashtag)):
                # Get matrix indices for the users
                user_i_idx = users_with_hashtag[i] - 1  # subtract 1 since array indices start from 0
                user_j_idx = users_with_hashtag[j] - 1
                
                # Check if the indices are within bounds
                if 0 <= user_i_idx < 138 and 0 <= user_j_idx < 138:
                    # Set the matrix entries for this pair of users to 1
                    matrix[user_i_idx][user_j_idx] = 1
                    matrix[user_j_idx][user_i_idx] = 1  # Symmetrical matrix
                else:
                    print(f"WARNING: Detected out of bounds OFFICE2 value. Skipping pair ({users_with_hashtag[i]}, {users_with_hashtag[j]})")
                
    # Convert the matrix to a DataFrame
    return pd.DataFrame(matrix, index=users, columns=users)


def save_to_sqlite(df, db_name, table_name):
    print(f"Saving DataFrame to SQLite with table name: {table_name}")  # Print current saving status
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='replace')
    conn.close()

# Create a dictionary mapping DataFrame names to DataFrames
df_dict = {
    'econ': econ,
    'env': env,
    'health': health,
    'imm': imm,
    'sjid': sjid,
    'sjlab': sjlab,
    'sjwom': sjwom
}

for name, df in df_dict.items():
    print(f"Generating adjacency matrix for DataFrame: {name}")  # Print current DataFrame being processed
    adjacency_df = generate_adjacency(df)
    save_to_sqlite(adjacency_df, "csvyerine2.db", name)  # Use the name directly as the table name

print("All done!")  # Print completion message


Generating adjacency matrix for DataFrame: econ
Processing cluster: ShopSmall
Processing cluster: DemsCreateJobs
Processing cluster: gasprices
Processing cluster: SmallBusinessSaturday
Processing cluster: CancelStudentDebt
Processing cluster: ManufacturingDay
Processing cluster: ChildTaxCredit
Processing cluster: InflationReductionAct
Processing cluster: BidenFlation
Processing cluster: inflation
Processing cluster: SmallBizSatuday
Processing cluster: FarmBill
Processing cluster: SmallBiz
Processing cluster: ShopLocal
Processing cluster: BidenRecession
Processing cluster: SocialSecurity
Processing cluster: Bidenomics
Processing cluster: AmericanRescuePlan
Processing cluster: IRS
Processing cluster: SPR
Processing cluster: jobs
Saving DataFrame to SQLite with table name: econ
Generating adjacency matrix for DataFrame: env
Processing cluster: DARTMission
Processing cluster: ForestProductsWeek
Processing cluster: TreeOfLife
Processing cluster: energy
Processing cluster: CleanWater50
Proce